In [ ]:
import random
import numpy as np

# Definição da classe Gridworld

In [ ]:
# definição da classe que caracteriza o grid no qual o agente atuará
class Gridworld:
    # o método construtor recebe o número de linhas 'n' e o número de colunas 'm' do grid
    def __init__(self, n, m):
        self.grid = [[0 for _ in range(m)] for _ in range(n)] # cria uma matriz com n linhas e m colunas

        # inicializa as listas para armazenar as coordenadas das montanhas e
        # das areias movediças
        self.coordenadas_montanhas = []
        self.coordenadas_areias = []

        # inicializa os pontos de inicio e chegada que serão definidos pelo usuário
        self.inicio = None
        self.chegada = None

        # inicializa uma matriz que armazenará as recompensas de cada ponto do grid
        self.recompensas = [[0 for _ in range(m)] for _ in range(n)]

    # método que permite definir as coordenadas das montanhas e areas
    def definir_obstaculos(self, coordenadas_montanhas, coordenadas_areias):
        # para cada coordenada (x, y) na lista 'coordenadas_montanhas', marca-se
        # -1 no grid
        self.coordenadas_montanhas = coordenadas_montanhas
        for x_montanha, y_montanha in coordenadas_montanhas:
            self.grid[y_montanha][x_montanha] = -1

        # para cada coordenada (x, y) na lista 'coordenadas_areias', marca-se
        # -2 no grid
        self.coordenadas_areias = coordenadas_areias
        for x_areia, y_areia in coordenadas_areias:
            self.grid[y_areia][x_areia] = -2

        # -1 e -2 foram escolhidos de maneira arbitraria, apenas para representar
        # os pontos que são obstáculos

    # método que permite definir os pontos de inicio e chegada,
    # que definem onde o agente iniciará e onde ele deve terminar
    def definir_inicio_chegada(self, inicio, chegada):
        self.inicio = inicio
        self.chegada = chegada
        x_inicio, y_inicio = inicio
        x_chegada, y_chegada = chegada
        self.grid[y_inicio][x_inicio] = -3
        self.grid[y_chegada][x_chegada] = -4

        # apenas marca um -3 na coordenada do inicio e -4 na coordenada
        # de chegada. valores escolhidos de maneira arbitria apenas para
        # diferenciar os pontos no grid

    # método que permite definir as recompensas para cada ponto do grid
    def definir_recompensas(self, recompensa_chegada, recompensa_areia, recompensa_montanha, recompensa_padrao):
        # inicialmente inicializa todos os pontos do grid com uma recompensa padrão
        for y in range(len(self.grid)):
            for x in range(len(self.grid[0])):
                self.recompensas[y][x] = recompensa_padrao

        # atualiza a recompensa do ponto de chegada com o valor recebido
        x_chegada, y_chegada = self.chegada
        self.recompensas[y_chegada][x_chegada] = recompensa_chegada

        # atualiza a recompensa para os pontos que são montanhas ou areias com os valores
        # recebidos
        for x_areia, y_areia in self.coordenadas_areias:
            self.recompensas[y_areia][x_areia] = recompensa_areia
        for x_montanha, y_montanha in self.coordenadas_montanhas:
            self.recompensas[y_montanha][x_montanha] = recompensa_montanha

    # permite visualizar o grid de duas maneiras:
    # 1. com cada ponto do grid sendo representado com um símbolo específico
    # 2. com cada ponto do grid sendo representado pela recompensa atribuida a esse ponto
    # obs: caso seja desejado visualizar a posição do agente é possível passar uma coordenada com a posição dele,
    # e nessa coordenada será colocado um 'X'
    def visualizar_grid(self, modo="simbolos", pos_agente=None):
        # se for desejado visualiza a partir de símbolos, cria-se um dicionário
        # que mapeia os números atribuídos a cada ponto do grid para uma letra
        # que representa o que cada ponto é
        if modo == "simbolos":
            simbolos = {
                0:  '   ',
                -1: ' M ', # M - montanha
                -2: ' A ', # A - areia
                -3: ' I ', # I - inicio
                -4: ' C ' # C - chegada
            }
            # percorre cada posição do grid. se a posição na iteração atual corresponder a posição do agente passada como parâemtro,
            # coloca um X nessa posição. se a posição atual não é a do agente, busca-se o valor dessa posição utilizando 'self.grid[y][x]'
            # e busca o símbolo correspondente no dicionário definido acima
            for y in range(len(self.grid)):
                for x in range(len(self.grid[y])):
                    if pos_agente == (x, y):
                        print(' X |', end="")
                    else:
                        print(f"{simbolos.get(self.grid[y][x], ' ? ')}|", end="")
                print("\n" + "----" * len(self.grid[0]))
        # se o modo de visualização for a partir das recompensas das posição, apenas itera em todas as posições
        # e pega o valor da recompensa de cada posição na matriz "recompensas"
        elif modo == "recompensas":
            for y in range(len(self.recompensas)):
                for x in range(len(self.recompensas[y])):
                    print(f"{self.recompensas[y][x]: 3} |", end="")
                print("\n" + "-----" * len(self.recompensas[0]))

# Definição da classe Agente

In [ ]:
#define um Agente que pode navegar dentro de um ambiente Gridworld
class Agente:
    # método construtor recebe a posição inicial do agente
    def __init__(self, inicio_pos):

        # armazena a posição inicial recebida
        self.inicio_pos = inicio_pos

        # armazena a posição atual do agente, que inicialmente é igual a
        # posição inicial
        self.posicao = inicio_pos

        # define as ações do agente, junto com a mudança nas coordenadas que cada
        # ação ira causar
        self.acoes = {'up': (0, -1), 'down': (0, 1), 'left': (-1, 0), 'right': (1, 0)}

    # método que permite o agente mover-se dentro do gridworld recebido como argumento.
    def mover(self, acao, gridworld):

        # verifica-se se a ação recebida é valida
        if acao not in self.acoes:
            # print(f"Ação '{acao}' não é válida.")
            return False

        # calcula a nova posição do agente somando a mudança de posição
        # (dx, dy) correspondente com a ação tomada
        dx, dy = self.acoes[acao]
        x, y = self.posicao
        nova_pos = (x + dx, y + dy)

        # verifica se a nova posição é válida, ou seja, se está dentro dos limites
        # do grid
        x_new, y_new = nova_pos
        if x_new < 0 or x_new >= len(gridworld.grid[0]) or y_new < 0 or y_new >= len(gridworld.grid):
            # print("Movimento fora dos limites do grid.")
            return False

        # verifica se a nova posição é uma montanha ou areia.
        # se for montanha, o agente permanecerá na posição atual
        # se for areia, o agente reiniciará, voltando para a posição inicial
        # definida no momento de inicialização do agente
        tipo_celula = gridworld.grid[nova_pos[1]][nova_pos[0]]
        if tipo_celula == -1:
            # print("Movimento bloqueado por uma montanha! Permanecendo na posição atual.")
            return False
        elif tipo_celula == -2:
            # print("Capturado por areia movediça! Reiniciando...")
            self.posicao = self.inicio_pos
            return False

        self.posicao = nova_pos
        return True

    # retorna a posição atual do agente no grid
    def obter_posicao(self):
        return self.posicao

    # retorna a lista com as ações possívels do agente, ou seja,
    # ['up', 'down', 'left', 'right']
    def obter_acoes(self):
        return list(self.acoes.keys())

# Definição da classe GeradorRotas

In [ ]:
# classe que gerencia as rotas percorridas por um agente em um gridworld
class GeradorRotas:
    # método construtor recebe um gridworld e um agente
    def __init__(self, gridworld, agente):
        self.gridworld = gridworld
        self.agente = agente

        # incialização do dicionário que armazenará as rotas cadastradas
        self.rotas = {}

    # cadastra uma nova rota se ela ainda não existir
    def cadastrar_rota(self, nome_rota):
        if nome_rota not in self.rotas:
            # a rota é basicamente uma lista de coordenadas. toda rota
            # é inicializada com a primeira coordenada sendo a posição inicial
            # do agente.
            self.rotas[nome_rota] = [self.agente.inicio_pos]
        else:
            print(f"A rota '{nome_rota} já existe.")

    # atualiza uma rota existente
    def atualizar_rota(self, nome_rota):
        # se a rota que deseja-se atualizar existir, basicamente
        # adiciona-se nela a posição atual do agente
        if nome_rota in self.rotas:
            self.rotas[nome_rota].append(self.agente.posicao)
        else:
            print(f"A rota '{nome_rota}' não foi encontrada.")

    # exibe as coordenadas de uma rota cadastrada
    def exibir_rota(self, nome_rota):
        if nome_rota in self.rotas:
            print(f"Rota '{nome_rota}': {self.rotas[nome_rota]}")
        else:
            print(f"A rota '{nome_rota}' não foi encontrada.")

    # reseta todas as rotas cadastradas, limpandno o dicionário 'rotas'
    def resetar_rotas(self):
        self.rotas = {}
        # print("Todas as rotas foram resetadas.")

    # move o agente aleatoriamente dentro do gridworld e atualiza a rota
    # correspondente.
    def mover_aleatoriamente(self, nome_rota):
        if nome_rota not in self.rotas:
            print(f"A rota '{nome_rota}' não existe.")
            return
        # seleciona uma ação aleatória dentro do conjunto de ações possíveis do agente
        acao = random.choice(self.agente.obter_acoes())

        # move o agente com a ação aleatória escolhida
        self.agente.mover(acao, self.gridworld)

        # adiciona a nova posição do agente a rota.
        # obs: caso a ação tomada leve o agente a posição de uma montanha,
        # ele permacenerá na mesma posição e será adicionado na lista da rota
        # a coordenada atual dele
        self.atualizar_rota(nome_rota)

    # gera uma rota aleatória que termina quando atinge-se um máximo de movimentos
    # ou quando o agente chega ao ponto de chegada
    def gerar_rota_aleatoria(self, nome_rota, max_movimentos=100):
        self.agente.posicao = self.agente.inicio_pos

        # cadastra a rota
        if nome_rota not in self.rotas:
            self.cadastrar_rota(nome_rota)

        movimentos = 0
        # enquanto o agente ainda poder se movimentar
        while movimentos < max_movimentos:
            # o agente move-se de maneira aleatória
            self.mover_aleatoriamente(nome_rota)
            # se a nova posição do agente após a ação for o ponto de chegada
            # encerra-se a execução e retorna-se o número de movimentos realizados
            if self.agente.obter_posicao() == self.gridworld.chegada:
                # print(f"Chegada alcançada em {movimentos + 1} movimentos na rota '{nome_rota}'.")
                self.agente.posicao = self.agente.inicio_pos
                return movimentos
            movimentos += 1

        # print("Limite de movimentos atingido.")
        self.agente.posicao = self.agente.inicio_pos
        return movimentos

    def definicao_valores_estados(self, fator_desconto, num_iteracoes=100):
      # matriz para armazenar o valor estimado para cada estado no grid.
      # inicialmente todos os valores são 0
      valores = [[0 for _ in range(len(self.gridworld.grid[0]))] for _ in range(len(self.gridworld.grid))]

      for _ in range(num_iteracoes):
          # matriz criada a cada iteração para armazenar os valores atualizados dos estados
          valores_atualizados = [[0 for _ in range(len(self.gridworld.grid[0]))] for _ in range(len(self.gridworld.grid))]

          # define-se o valor do estado de chegada coomo sendo sua recompensa direta
          # sem nenhum desconto aplicado
          x_chegada, y_chegada = self.gridworld.chegada
          valores_atualizados[y_chegada][x_chegada] = self.gridworld.recompensas[y_chegada][x_chegada]

          # iteração em todos os estados (posições no grid)
          for y in range(len(self.gridworld.grid)):
              for x in range(len(self.gridworld.grid[0])):
                  # se a posição atual for a posição de chegada, não deseja-se realizar nenhuma alteraçõa
                  # pois o valor dessa posição já foi definido como sua recompensa
                  if (x, y) == self.gridworld.chegada:
                      continue

                  # se a posição atual for uma montenha ou areia movedição, o valor do estado é 0,
                  # o que reflete o fato de querermos que o agente não vá para esses estados
                  if self.gridworld.grid[y][x] == -1 or self.gridworld.grid[y][x] == -2:
                      valores_atualizados[y][x] = 0

                  else:
                      # defini-se o valor máximo
                      valor_maximo_vizinho = float('-inf')

                      # para cada ação possível do agente:
                      # 1. calcula-se a nova posição (novo_x, novo_y) resultante da ação
                      # 2. verifica-se se a nova posição está dentro dos limites do grid
                      # 3. se a nova posições estiver dentro dos limites, calcula-se o valor
                      # descontado do estado vizinho
                      # 4. compara-se o valor descontado do estado vizinho com o valor máximo já
                      # encontrado para o vizinho
                      for acao, (dx, dy) in self.agente.acoes.items():
                          novo_x, novo_y = x + dx, y + dy
                          if 0 <= novo_x < len(self.gridworld.grid[0]) and 0 <= novo_y < len(gridworld.grid):
                              valor_vizinho = fator_desconto * valores[novo_y][novo_x]
                              valor_maximo_vizinho = max(valor_maximo_vizinho, valor_vizinho)

                      valores_atualizados[y][x] = valor_maximo_vizinho

          # atualização da matriz com os valores de cada estado, após cada iteração
          valores = valores_atualizados

      return valores

    def gerar_rota_inteligente(self, nome_rota, valores_estados):
        # registr a a nova rota
        self.cadastrar_rota(nome_rota)

        movimentos = 0
        # enquanto o agente ainda poder se movimentar
        while movimentos < 100:
            # obtem a posição atual do agente
            posicao_atual = self.agente.obter_posicao()
            x, y = posicao_atual

            # se a posição atual for a posição de chegada,
            # encerra a execução
            if posicao_atual == self.gridworld.chegada:
                print(f"Chegada alcançada em {movimentos} movimentos na rota '{nome_rota}'.")
                self.agente.posicao = self.agente.inicio_pos
                return


            melhor_acao = None
            melhor_valor = float('-inf')

            # itera em cada possível ação definida no agente, calculando a nova
            # posição que resultaria dessa ação e verificando se essa nova posição
            # é válida. para cada posição válida, pega o valor de estado dessa posição
            # e verifica se ele é maior do que o maior valor encontrado até o momento.
            # se for, atualiza o maior valor.
            for acao, (dx, dy) in self.agente.acoes.items():
                novo_x, novo_y = x + dx, y + dy
                if 0 <= novo_x < len(self.gridworld.grid[0]) and 0 <= novo_y < len(self.gridworld.grid):
                    valor_vizinho = valores_estados[novo_y][novo_x]
                    if valor_vizinho > melhor_valor:
                        melhor_acao = acao
                        melhor_valor = valor_vizinho

            # se a melhor ação (a que proporciona ir para um estado com maior valor) for encontrada
            # então toma essa ação
            if melhor_acao:
                self.agente.mover(melhor_acao, self.gridworld)
                self.atualizar_rota(nome_rota)
            else:
                print("Não foi possível encontrar uma ação válida. Reiniciando...")
                self.agente.posicao = self.agente.inicio_pos

            movimentos += 1

        print("Limite de movimentos atingido.")
        self.agente.posicao = self.agente.inicio_pos

# Execução principal

## Definição gridworld

In [ ]:
gridworld = Gridworld(8, 8)

## Definição dos pontos de inicio e fim

In [ ]:
inicio = (0, 0)
chegada = (7, 6)

gridworld.definir_inicio_chegada(inicio, chegada)

## Definição dos obstáculos

In [ ]:
coordenadas_montanhas = [(0, 1), (3, 4), (3, 7)]
coordenadas_areias = [(0, 5), (5, 5), (7, 2)]

gridworld.definir_obstaculos(coordenadas_montanhas, coordenadas_areias)

## Definição das recompensas

In [ ]:
recompensa_montanha = -1
recompensa_areia = -1
recompensa_padrao = 0
recompensa_chegada = 1

gridworld.definir_recompensas(recompensa_chegada, recompensa_areia, recompensa_montanha, recompensa_padrao)

## Definição do agente

In [ ]:
agente = Agente(inicio)

## Visualização do grid

### Modo símbolos

In [ ]:
gridworld.visualizar_grid(modo="simbolos")

 I |   |   |   |   |   |   |   |
--------------------------------
 M |   |   |   |   |   |   |   |
--------------------------------
   |   |   |   |   |   |   | A |
--------------------------------
   |   |   |   |   |   |   |   |
--------------------------------
   |   |   | M |   |   |   |   |
--------------------------------
 A |   |   |   |   | A |   |   |
--------------------------------
   |   |   |   |   |   |   | C |
--------------------------------
   |   |   | M |   |   |   |   |
--------------------------------


### Modo recompensas

In [ ]:
gridworld.visualizar_grid(modo="recompensas")

  0 |  0 |  0 |  0 |  0 |  0 |  0 |  0 |
----------------------------------------
 -1 |  0 |  0 |  0 |  0 |  0 |  0 |  0 |
----------------------------------------
  0 |  0 |  0 |  0 |  0 |  0 |  0 | -1 |
----------------------------------------
  0 |  0 |  0 |  0 |  0 |  0 |  0 |  0 |
----------------------------------------
  0 |  0 |  0 | -1 |  0 |  0 |  0 |  0 |
----------------------------------------
 -1 |  0 |  0 |  0 |  0 | -1 |  0 |  0 |
----------------------------------------
  0 |  0 |  0 |  0 |  0 |  0 |  0 |  1 |
----------------------------------------
  0 |  0 |  0 | -1 |  0 |  0 |  0 |  0 |
----------------------------------------


 ## Definição do gerador de rotas

In [ ]:
gerador_rotas = GeradorRotas(gridworld, agente)

## Geração de rotas aleatórias

Gerando 100 rotas aleatórias para comparar quantos movimentos em média leva-se para o agente chegar até o ponto de chegada a partir do ponto de início definidos.
Os prints de cada rota foram comentados, pois eles servem apenas para debug, podendo ser ignorados para o objetivo dessa análise. Caso deseje visualizar os prints, basta descomentá-los na classe Agente e GeradorRotas

In [ ]:
num_movimentos = []
num_experimentos = 100
maximo_movimentos_permitido = 10**4 # define que o agente pode percorrer no máximo 10000 movimentos

for _ in range(num_experimentos):
  qtd_movimentos = gerador_rotas.gerar_rota_aleatoria("rota_aleatoria", maximo_movimentos_permitido)
  num_movimentos.append(qtd_movimentos)
  gerador_rotas.resetar_rotas()

In [ ]:
np.mean(num_movimentos)

1225.14

Em média, de maneira aleatória, o agente precisa realziar 1225 movimentos para chegar ao ponto (7,6), saindo do ponto (0,0).

## Geração de rotas com base nos valores de estado

### Definindo e visualizando os valores de estado

In [ ]:
valores_estado = gerador_rotas.definicao_valores_estados(fator_desconto=0.9)

In [ ]:
for linha in valores_estado:
    for valor in linha:
        print("{:.2f}".format(valor), end=" ")
    print()

0.25 0.28 0.31 0.35 0.39 0.43 0.48 0.43 
0.00 0.31 0.35 0.39 0.43 0.48 0.53 0.48 
0.31 0.35 0.39 0.43 0.48 0.53 0.59 0.00 
0.35 0.39 0.43 0.48 0.53 0.59 0.66 0.73 
0.39 0.43 0.48 0.00 0.59 0.66 0.73 0.81 
0.00 0.48 0.53 0.59 0.66 0.00 0.81 0.90 
0.48 0.53 0.59 0.66 0.73 0.81 0.90 1.00 
0.43 0.48 0.53 0.00 0.66 0.73 0.81 0.90 


### Gerando rota inteligente

In [ ]:
gerador_rotas.gerar_rota_inteligente("rota_inteligente", valores_estado)

Chegada alcançada em 13 movimentos na rota 'rota_inteligente'.


In [ ]:
gerador_rotas.exibir_rota("rota_inteligente")

Rota 'rota_inteligente': [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6), (6, 6), (7, 6)]


Com a rota inteligente, que é baseamente na tomada da ação que leva a um estado com maior valor, o agente precisa de apenas 13 movimentos para chegar no ponto (7, 6), partindo do ponto (0, 0).